In [1]:
from exp.Models import *
from exp.Preprocessing import *

## Preprocessing

In [2]:
%%time
train_df, test_df = preprocess_and_train_test_split(URLs.BPIC_2012,cache=True,override=True)
train_df.head(5)

# duration to next event zu relative timestamp, plus spalte next relative timestamp hinzufügen

Successfully created the directory ./.cache/datasets/BPI_Challenge_2012
CPU times: user 39.4 s, sys: 759 ms, total: 40.1 s
Wall time: 39.8 s


,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp,REG_DATE,AMOUNT_REQ,activity:name,activity,resource,relative timestamp,next relative timestamp,duration to end,next activity,next resource,last resource,outcome,activity suffix,resource suffix
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_SUBMITTED_COMPLETE,11,54,0.334,53.360,1072732.480,8,54,8,34,"[8, 9, 26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 3...","[54, 54, 54, 70, 15, 15, 15, 15, 15, 70, 70, 7..."
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_PARTLYSUBMITTED_COMPLETE,8,54,53.026,54.329,1072732.146,9,54,8,34,"[9, 26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 31, ...","[54, 54, 70, 15, 15, 15, 15, 15, 70, 70, 70, 7..."
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-10-01 00:38:44.546000+02:00,20000,A_PREACCEPTED_COMPLETE,9,54,0.969,39481.891,1072679.120,26,54,8,34,"[26, 27, 2, 16, 7, 14, 18, 32, 25, 33, 31, 33,...","[54, 70, 15, 15, 15, 15, 15, 70, 70, 70, 70, 2..."
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-10-01 00:38:44.546000+02:00,20000,W_Completeren aanvraag_SCHEDULE,26,54,39427.562,39838.762,1072678.151,27,70,8,34,"[27, 2, 16, 7, 14, 18, 32, 25, 33, 31, 33, 31,...","[70, 15, 15, 15, 15, 15, 70, 70, 70, 70, 24, 2..."
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-10-01 00:38:44.546000+02:00,20000,W_Completeren aanvraag_START,27,70,356.871,39984.697,1033250.589,2,15,8,34,"[2, 16, 7, 14, 18, 32, 25, 33, 31, 33, 31, 33,...","[15, 15, 15, 15, 15, 70, 70, 70, 70, 24, 24, 4..."


In [3]:
test_df[["relative timestamp", "next relative timestamp"]][test_df["activity"] == 11]

,relative timestamp,next relative timestamp
203,0.294,41.604
949,0.177,42.554
1155,0.174,43.258
1217,0.323,49.811
1342,0.168,40.677
...,...,...
261828,0.721,37.120
261909,0.432,16.884
261936,0.222,43.717
261954,0.254,43.800


In [4]:
#g = test_df.groupby("trace_id")

In [5]:
#def predict_results(g, index, clf, col):
#    tmp = g.apply(lambda x: x[col].values[index:-1])
#    tmp = tmp[tmp.apply(lambda x: len(x)>0)]
#    for i in tmp:
#        print(i[2:-1])
#        break
#    print(tmp)
#    res = tmp.apply(lambda x: clf.predict(x[index:-1]))
#    return res

In [6]:
# clf = RandomForestClassifier()
# predict_results(g, 2, clf, ["activity"])

In [7]:
# from datetime import datetime
# 
# times = train_df[["trace_id", "time:timestamp"]]
# array = [t.timestamp() for t in times["time:timestamp"]]
# # array2 = [t.astype("float") for t in times["time:timestamp"].values]
# 
# array[:3]

In [8]:
# array[2] - array[1]

In [9]:
#test_df[["relative timestamp", "next relative timestamp"]][test_df["trace_id"] == "173709"]

In [10]:
def create_traces(event_df,trace_id='trace_id'):
    ll=[]
    trace_ids=[]
    cols=list(event_df)
    cols.remove(trace_id)
    for n, g in event_df.groupby(trace_id):
        l=[]
        
        for c in cols:
            l.append(list(g[c]))
        ll.append(l)
        trace_ids.append(n)  
        

    df=pd.DataFrame(ll,columns=cols)
    df["trace_id"]=trace_ids
    return df

kopie = test_df.copy()
group = create_traces(kopie)

def process_data_for_suffix_prediction(test,input_cols=None,output_col=8,startIndex=2):
    xs,ys=[],[]
    if input_cols == None: input_cols=list(test)
    input_cols=listify(input_cols)
    for trace in test.values:
        for i in range(startIndex,len(listify(trace[0]))):
            x,y=[],[]
            for c in range(len(input_cols)):
                x.append(trace[c][:i])
            
            xs.append(x)
            ys.append(trace[output_col][i:])
    return pd.DataFrame(xs,columns=input_cols),ys

#xs, ys = process_data_for_suffix_prediction(group)
#xs[:3]

In [11]:
#max(map(lambda x: len(x), xs["activity"]))

In [12]:
#xs.head(10)

## Evaluation Functions

In [13]:
def evaluate_outcome_prediction(pred,ground_truth): return flat_accuracy(pred,ground_truth)
def flat_accuracy(pred,ground_truth):
    x=np.array([j for i in pred.values for j in i])
    y=np.array([j for i in ground_truth.values for j in i])
    return float((x==y).mean()) if len(x)==len(y) else None

def evaluate_next_step_prediction(pred,ground_truth): 
    #print(len(pred[0]), len(ground_truth[0]))
    #print(pred[0], ground_truth[0])
    return flat_accuracy(pred,ground_truth)
def evaluate_next_resource_prediction(pred,ground_truth): 
    #print(len(pred[0]), len(ground_truth[0]))
    #print(pred[0], ground_truth[0])
    return flat_accuracy(pred,ground_truth)
def evaluate_last_resource_prediction(pred,ground_truth): return flat_accuracy(pred,ground_truth)
def dl_measure(pred,ground_truth):
    pred=pred.values
    ground_truth=ground_truth.values
    scores=[]
    if len(pred)!=len(ground_truth): return None
    for i in range(len(pred)):
        if len(pred[i])!=len(ground_truth[i]): return None
        for j in range(len(pred[i])):
            scores.append(1-normalized_damerau_levenshtein_distance(pred[i][j], ground_truth[i][j]))
    return np.mean(scores)
                    
def evaluate_activity_suffix_prediction(pred,ground_truth): return 1 # dl_measure(pred,ground_truth)
def evaluate_resource_suffix_prediction(pred,ground_truth): return 1 # dl_measure(pred,ground_truth)

def mse(output, targ): 
    if not torch.is_tensor(output): output=tensor(output)
    if not torch.is_tensor(targ): targ=tensor(targ)
    return (output.squeeze(-1) - targ).pow(2).mean()

def flat_mse(pred,ground_truth):
    x=np.array([j for i in pred.values for j in i])
    y=np.array([j for i in ground_truth.values for j in i])    
    return float(mse(x,y)) if len(x)==len(y) else None

def evaluate_duration_to_next_event_prediction(pred,ground_truth): 
    print("Pred: {}".format(pred))
    print("GroundTruth: {}".format(ground_truth))
    return flat_mse(pred,ground_truth)
def evaluate_duration_to_end_prediction(pred,ground_truth): return flat_mse(pred,ground_truth)

## Models

### Perfect Model (groundtruth)

In [14]:
model=Perfect_PPM()
o=model.predict(test_df)
len(o)

8

In [15]:
model.next_step_prediction(test_df)

trace_id
173718    [26, 27, 2, 7, 16, 14, 18, 32, 25, 33, 31, 33,...
173793    [26, 27, 25, 27, 2, 16, 7, 14, 18, 32, 25, 33,...
173835    [26, 27, 25, 27, 25, 27, 2, 16, 7, 14, 18, 32,...
173862                                          [21, 6, 19]
173898    [21, 9, 26, 19, 27, 25, 27, 25, 27, 2, 7, 16, ...
                                ...                        
214256                                      [26, 27, 5, 25]
214286    [26, 27, 25, 27, 2, 16, 7, 14, 18, 32, 25, 33,...
214301                                  [21, 19, 21, 6, 19]
214310    [26, 27, 25, 27, 25, 27, 2, 7, 16, 14, 18, 32,...
214313    [26, 27, 25, 27, 2, 7, 16, 14, 18, 32, 25, 33,...
Length: 982, dtype: object

In [16]:
model.activity_suffix_prediction(test_df)

trace_id
173718    [[26, 27, 2, 7, 16, 14, 18, 32, 25, 33, 31, 33...
173793    [[26, 27, 25, 27, 2, 16, 7, 14, 18, 32, 25, 33...
173835    [[26, 27, 25, 27, 25, 27, 2, 16, 7, 14, 18, 32...
173862                         [[21, 6, 19], [6, 19], [19]]
173898    [[21, 9, 26, 19, 27, 25, 27, 25, 27, 2, 7, 16,...
                                ...                        
214256        [[26, 27, 5, 25], [27, 5, 25], [5, 25], [25]]
214286    [[26, 27, 25, 27, 2, 16, 7, 14, 18, 32, 25, 33...
214301    [[21, 19, 21, 6, 19], [19, 21, 6, 19], [21, 6,...
214310    [[26, 27, 25, 27, 25, 27, 2, 7, 16, 14, 18, 32...
214313    [[26, 27, 25, 27, 2, 7, 16, 14, 18, 32, 25, 33...
Length: 982, dtype: object

In [17]:
model.duration_to_next_event_prediction(test_df)

trace_id
173718    [69.091, 282.614, 421.36199999999997, 421.3619...
173793    [136498.315, 136626.116, 155081.125, 155835.43...
173835    [84857.117, 85003.175, 85802.57400000001, 8580...
173862    [77127.30699999999, 77131.39099999999, 77131.3...
173898    [63086.151000000005, 63086.27300000001, 63087....
                                ...                        
214256    [74649.096, 74753.24500000001, 74755.012000000...
214286    [49.471, 354.553, 62806.968, 63156.542, 63346....
214301    [45045.009999999995, 46177.581999999995, 46358...
214310    [68720.342, 68802.176, 76457.335, 76629.078000...
214313    [63877.332, 64072.154, 66979.43400000001, 6720...
Length: 982, dtype: object

In [18]:
model.duration_to_end_prediction(test_df)

trace_id
173718    [2241580.823, 2241580.387, 2241553.336, 224133...
173793    [1439502.931, 1439502.624, 1303047.17, 1302919...
173835    [1396286.553, 1396286.187, 1311479.247, 131133...
173862                           [77090.714, 82.052, 4.084]
173898    [964496.801, 901607.064, 901422.859, 901422.73...
                                ...                        
214256               [74717.751, 74717.398, 105.916, 1.767]
214286    [1025316.443, 1025315.873, 1025304.092, 102499...
214301      [46345.053, 2653.766, 1316.927, 184.355, 3.634]
214310    [833141.283, 833140.931, 764464.658, 764382.82...
214313    [830519.022, 830518.592, 766685.49, 766490.668...
Length: 982, dtype: object

In [19]:
model.train('test')

### Most-frequent-value Model

### RandomForest Model

In [20]:
# from sklearn.ensemble import RandomForestRegressor
# 
# copy = train_df.copy()
# copy["duration to next event"].fillna(0.0, inplace=True)
# copy["time:timestamp"] = [i.astype("float") for i in copy["time:timestamp"].values]
# 
# clf = RandomForestRegressor(n_estimators=10)
# train = copy[["activity", "time:timestamp"]]
# target = copy["duration to next event"]           # naives duration to next event training
# clf.fit(train, target)
# 
# test_copy = test_df.copy()
# test_copy["time:timestamp"] = [i.astype("float") for i in test_copy["time:timestamp"].values]
# 
# preds = clf.predict(test_copy[["activity", "time:timestamp"]])
# print(preds[:3])

### LogisticRegression Model

#### Tests

In [21]:
#%%time
#from datetime import datetime

#traindata = train_df.copy()
### Time to next event prediction:
#X = [0] *len(traindata)
#for date in traindata["time:timestamp"]:
#    for i in range(len(traindata)):
#        X[i] = datetime.strptime(str(date), "%Y-%m-%d %H:%M:%S.%f%z")

#y = traindata["duration to next event"]

### Trying to group data for training does not work...
#X = traindata.groupby("trace_id", sort=False)["activity"]
#y = traindata.groupby("trace_id", sort=False)["next activity"]

#X = traindata["activity"].to_numpy().reshape(-1,1)
#y = traindata["next activity"].to_numpy().reshape(-1,1)

#clf = LogisticRegression()
#clf.fit(X, y)

#all_traintraces = []
#for trace, values in X:
 #   all_traintraces.append(list(X["activity"].get_group(trace)))
    
#clf.fit(all_traintraces,)
    
#print(all_traces[:3])

In [22]:
#group = train_df.groupby("trace_id")
#traces = get_traces(group, "activity")
#traces[:2]

In [23]:
#traini = train_df.copy()
#Xtraini = traini["activity"].to_numpy().reshape(-1,1)
#ytraini = traini["next activity"]

#logreg = LogisticRegression(solver="sag", multi_class="multinomial", max_iter=4000)
#logreg.fit(Xtraini, ytraini)
#logreg.n_iter_

In [24]:
#testdata = test_df.copy()
#Xtest = testdata["activity"].to_numpy().reshape(-1,1)
#ytest = testdata["next activity"]

#logreg.predict(Xtest)
#logreg.score(Xtest, ytest)

In [25]:
#ytest = testdata["next activity"].to_numpy().reshape(-1,1)
#clf.score(Xtest, ytest)

#### Modell:

In [26]:
#class LogReg_default_PPM(PPModel): 
#    def __init__(self):
#        self.clf1 = LogisticRegression(solver="sag", multi_class="multinomial")
#        self.clf2 = LogisticRegression(solver="sag", multi_class="multinomial")
#        self.clf3 = LogisticRegression(solver="sag", multi_class="multinomial")
#        self.clf4 = LogisticRegression(solver="sag", multi_class="multinomial")
#    
#    def name(self):
#        return self.__class__.__name__
#    
#    def train(self, data): 
#        train1 = data["activity"].to_numpy().reshape(-1,1) # naives next activity training
#        target1 = data["next activity"]
#        self.clf1.fit(train1, target1)
#        
#        train2 = data["resource"].to_numpy().reshape(-1,1) # naives next resource training
#        target2 = data["next resource"]
#        self.clf2.fit(train2, target2)
#        
#        train3 = data["resource"].to_numpy().reshape(-1,1) # naives last resource training
#        target3 = data["last resource"]
#        self.clf3.fit(train3, target3)
#        
#        train4 = data["activity"].to_numpy().reshape(-1,1) # naives outcome training
#        target4 = data["outcome"]
#        self.clf4.fit(train4, target4)
#        
#        return self.clf1, self.clf2, self.clf3, self.clf4
#    
#    def predict(self, test, index=2):
#        self.result_df = test.copy()
#        X1 = test["activity"].to_numpy().reshape(-1,1) # for nsp
#        X2 = test["resource"].to_numpy().reshape(-1,1) # for nrp
#        X3 = test["resource"].to_numpy().reshape(-1,1) # for lrp
#        X4 = test["activity"].to_numpy().reshape(-1,1) # for op
#        
#        self.result_df["nsp"] = self.clf1.predict(X1) # nsp
#        self.result_df["nrp"] = self.clf2.predict(X2) # nrp
#        self.result_df["lrp"] = self.clf3.predict(X2) # lrp
#        self.result_df["op"] = self.clf4.predict(X2) # op
#        
#        self.cases = self.result_df.groupby('trace_id')
#        return super().predict(test, index)
#    
#    def next_step_prediction(self, test, index=2): return PPModel._return_results(self, index, "nsp")
#    
#    def next_resource_prediction(self, test, index=2): return PPModel._return_results(self, index, "nrp")
#    
#    def last_resource_prediction(self, test, index=2): return PPModel._return_results(self, index, "lrp")
#    
#    def outcome_prediction(self, test, index=2):  return PPModel._return_results(self, index, "op")
#    
#    def duration_to_next_event_prediction(self, test, index=2): return pd.Series(dtype=float)
#    def duration_to_end_prediction(self, test, index=2):  return pd.Series(dtype=float)
#    def activity_suffix_prediction(self, test, index=2):  return pd.Series(dtype=float)
#    def resource_suffix_prediction(self, test, index=2):  return pd.Series(dtype=float)
#    def reset(self): pass

### Own LSTM

In [38]:
from exp.dl_utils import *
from exp.control_flow_prediction import *

# import torch
# import torchvision
# import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
# import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
# import torch.nn.functional as F # All functions that don't have any parameters
# from torch.utils.data import DataLoader # Gives easier dataset managment and creates mini batches
# import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
# import torchvision.transforms as transforms # Transformations we can perform on our dataset

# Set device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Hyperparameters
input_size = 175 # is varying in this case --> every trace differs in length --> longest = 175
hidden_size = 256
num_layers = 2
num_classes = 10 # depends on the task to perform nsp/nrp/lrp/...
sequence_length = 28
learning_rate = 0.005
batch_size = 1 # 64
num_epochs = 2

In [28]:
activity_classes = len(train_df["activity"].unique())
resource_classes = len(train_df["resource"].unique())
print(activity_classes, resource_classes)

36 69


In [45]:
train_traces = create_traces(train_df[["trace_id", "activity", "next activity"]])
test_traces = create_traces(test_df[["trace_id", "activity", "next activity"]])

In [46]:
# train statistics
train_trace_lengths = [len(trace) for trace in train_traces["activity"]]
long_train_trace = max(train_trace_lengths)
words_count = np.unique([j for i in train_traces["activity"] for j in i]).size

print(long_train_trace, words_count)

175 36


In [47]:
def pad_traces(traces, col, pad_len):
    for i in range(len(traces[col])):
        x = [0]*pad_len
        trace = traces[col][i]
        length = len(trace)
        x[0:length] = trace
        traces[col][i] = x
        return traces
    
train_traces = pad_traces(train_traces, "activity", 175)
test_traces = pad_traces(test_traces, "activity", 175)

In [ ]:
activity-spalte concatenieren fürs training


In [ ]:
#cf_list = ControlFlowList.from_df(train_df)

In [ ]:
#tp = TokenizeProcessor()
#' -> '.join(tp(cf_list[:2])[0])

In [ ]:
#proc_tok, proc_num = TokenizeProcessor(), NumericalizeProcessor()
#encoded_cf = label_by_func(cf_list, lambda x: 0, proc_x = [proc_tok, proc_num]) # statt cf_list muss sd aber unsere daten sind schon geteilt

#encoded_cf

In [ ]:
nn.LSTM??

In [48]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True) #bias, dropout, bidirectional
        self.fc = nn.Linear(hidden_size, num_classes) #hidden_size*sequence_length
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # define hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # define cell state
        
        #out, (hidden_state, cell_state)
        out, _ = self.lstm(x, (h0, c0)) # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # take all training batches at the same time, only the last hidden layer and all features:
        # this makes sense since the last hidden layer has got information of every hidden layer before
        out = self.fc(out[:, -1, :]) 
        return out

In [49]:
# Model
lstm = LSTM(input_size, hidden_size, num_layers, activity_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(lstm.parameters(), lr=learning_rate)

In [51]:
DataLoader??

In [50]:
train_loader = DataLoader(dataset=train_traces, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_traces, batch_size=batch_size, shuffle=True)

In [52]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
        
    num_correct = 0
    num_samples = 0
    
    # Set model to eval
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with \
              accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    # Set model back to train
    model.train()

KeyError: 2678

In [ ]:
# orientiert an get_language_model:
def build_dl_model(model, vocab_sz, emb_sz, n_hid, n_layers, pad_token, output_p=0.4, hidden_p=0.2, 
                   input_p=0.6, embed_p=0.1, weight_p=0.5, tie_weights=True, bias=True):
    
    dl_model = model(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = dl_model.emb if tie_weights else None
    return SequentialRNN(dl_model, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

# define SequentialRNN and LinearDecoder --> alles sehr kompliziert, erstmal einfacher machen?!

### LSTM (Tax)

In [ ]:
# from __future__ import print_function, division
# from keras.models import Sequential, Model
# from keras.layers.core import Dense
# from keras.layers.recurrent import LSTM, GRU, SimpleRNN
# from keras.layers import Input
# from keras.utils.data_utils import get_file
# from keras.optimizers import Nadam
# from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from keras.layers.normalization import BatchNormalization
# from collections import Counter
# import unicodecsv
# import numpy as np
# import random
# import sys
# import os
# import copy
# import csv
# import time
# from datetime import datetime
# from math import log

In [ ]:
# def create_traces(event_df, trace_id='trace_id'):
#     ll=[]
#     trace_ids=[]
#     cols=list(event_df)
#     cols.remove(trace_id)
#     for n, g in event_df.groupby(trace_id): #progress_bar()
#         l=[]
#         for c in cols:
#             l.append(list(g[c]))
#         ll.append(l)
#         trace_ids.append(n)  
#     df=pd.DataFrame(ll,columns=cols)
#     df["trace_id"] = trace_ids
#     return df
# 
# def drop_short_cases(traces, col, index=2):
#     return traces[traces[col].map(len) > (index+1)].reset_index(drop=True)
# 
# train = create_traces(train_df)[["activity", "resource"]]
# train = drop_short_cases(train, "activity")
# train.head()

In [ ]:
#train_df_copy = train_df.copy()
#lines = list(map(lambda x: x+'!',train_df_copy["activity"])) # put delimiter symbol
#target_chars = list(map(lambda x: set(x),lines))
# lines.remove('!')
#chars = np.unique(train_df_copy["activity"])
#len(chars)

In [ ]:
# maxlen = max(map(lambda x: len(x), train["activity"]))
# num_features = len(train.columns) # in deren implementierung: num_features = len(chars)+5
# # Building the model
# 
# 
# main_input = Input(shape=(maxlen, num_features), name='main_input')
# # train a 2-layer LSTM with one shared layer
# # the shared layer = l1, # the layer specialized in activity prediction = l2_1, the layer specialized in time prediction = l2_2
# l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input)
# b1 = BatchNormalization()(l1)
# l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1)
# b2_1 = BatchNormalization()(l2_1)
# l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1)
# b2_2 = BatchNormalization()(l2_2)
# act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
# time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)
# 
# model = Model(inputs=[main_input], outputs=[act_output, time_output])
# opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)
# 
# model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
# early_stopping = EarlyStopping(monitor='val_loss', patience=42)
# model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
# lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
# 
# model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)

## Runner:

In [ ]:
next_step_prediction_index=2
next_resource_prediction_index=2
last_resource_prediction_index=2
last_resource_prediction_index=2
outcome_prediction_index=2
duration_to_end_prediction_index=2
duration_to_next_event_prediction_index=2
suffix_prediction_index=2

# Überschreiben der eigentlichen funktionen, da diese den Kernel zum Abstürzen bringt
def evaluate_resource_suffix_prediction(pred, ground_truth):
    return 1

def evaluate_resource_suffix_prediction(pred, ground_truth):
    return 1

def evaluate_model(trained_model, testset, dataset_url, ground_truth):
    y_nsp, y_nrp, y_lrp, y_op, y_dtnep, y_dtep, y_asp, y_rsp = ground_truth
    pred_nsp, pred_nrp, pred_lrp, pred_op, pred_dtnep, pred_dtep, pred_asp, pred_rsp = trained_model.predict(testset)
    nsp = evaluate_next_step_prediction(pred_nsp, y_nsp)
    nrp = evaluate_next_resource_prediction(pred_nrp, y_nrp)
    lrp = evaluate_last_resource_prediction(pred_lrp, y_lrp)
    op = evaluate_outcome_prediction(pred_op, y_op)
    print("Trained Model: {}".format(trained_model))
    dtnep = evaluate_duration_to_next_event_prediction(pred_dtnep, y_dtnep)
    dtep = evaluate_duration_to_end_prediction(pred_dtep, y_dtep)
    asp = evaluate_activity_suffix_prediction(pred_asp, y_asp)
    rsp = evaluate_resource_suffix_prediction(pred_rsp, y_rsp)

    return [dataset_url.split('=')[-1].split('.')[0], trained_model.name(), nsp, nrp, lrp, op, dtnep, dtep, asp, rsp]

In [ ]:
_ground_truth_cols=[ 
 'next relative timestamp',
 'duration to end',
 'next activity',
 'next resource',
 'last resource',
 'outcome',
 'activity suffix',
 'resource suffix']

In [ ]:
cols = [x for x in list(test_df) if x not in _ground_truth_cols]

In [ ]:
_ground_truth_cols + ['trace_id']

In [ ]:
class Performance_Statistic():
    def __init__(self):
        self.df = pd.DataFrame(
        columns=['Dataset', 'Model', 'Next Step', 'Next Resource', 'Last Resource', 'Outcome',
                'Next relative Timestamp', 'Duration to Outcome', 'Activity Suffix', 'Resource Suffix'])
    def update(self,model_performance): self.df.loc[len(self.df)] = model_performance
    def to_df(self):
        return self.df

In [ ]:
def runner(dataset_urls, models, seed=42, cache_datasets=True, ground_truth_cols=_ground_truth_cols):
    np.random.seed(seed)
    performance_statistic = Performance_Statistic()
    for dataset_url in dataset_urls:
        train, test = preprocess_and_train_test_split(dataset_url, cache=cache_datasets)
        perfect_ppm = Perfect_PPM()
        cols = [x for x in list(test) if x not in ground_truth_cols]
        ground_truth = perfect_ppm.predict(test[ground_truth_cols + ['trace_id']])
        for model in models:
            model.train(train)
            model_performance = evaluate_model(model, test[cols], dataset_url, ground_truth)
            model.reset()
            performance_statistic.update(model_performance)
        
        performance_statistic.update(evaluate_model(perfect_ppm, test[ground_truth_cols + ['trace_id']], dataset_url, ground_truth))
        perfect_ppm.reset()
    return performance_statistic.to_df()

In [ ]:
dataset_urls = [URLs.BPIC_2012]
models = [MFV_PPM(), SimpleRandomForest_PPM(), RandomForest_PPM()] #, LogReg_default_PPM() 

In [ ]:
%%time
stats = runner(dataset_urls,models)
display(stats)

An Johannes:
Auswirkungen durch training auf ("activity", "resource") statt nur auf "resource" im SimpleRandomForest:
- Next Resource Prediction ging von 0.734754 auf 0.735531 hoch
- Last Resource Prediction ging von 0.258115 auf 0.318277 hoch

## Plotting

In [ ]:
import matplotlib.pyplot as plt

data = stats.values[:-1]

# styling of the plot
fig1 = plt.figure()
ax = fig1.add_axes([0,0,1,1])
ax.set_title("Illustration of Statistics")
ax.set_ylim(0.00, 1.00)
ax.yaxis.grid(True, which="major", linestyle=':')
ax.set_xticks([0.25, 1.25, 2.25, 3.25])
ax.set_xticklabels(["NSP", "NRP", "LRP", "OP"])

ax.bar(0.00, data[0][2], color = 'lightgrey', width = 0.25, label="MFV")
ax.bar(0.25, data[1][2], color = 'darkgrey', width = 0.25, label="SimpleRF")
ax.bar(0.50, data[2][2], color = 'orange', width = 0.25, label="WindowRF")
ax.legend()

ax.bar(1.00, data[0][3], color = 'lightgrey', width = 0.25)
ax.bar(1.25, data[1][3], color = 'darkgrey', width = 0.25)
ax.bar(1.50, data[2][3], color = 'orange', width = 0.25)

ax.bar(2.00, data[0][4], color = 'lightgrey', width = 0.25)
ax.bar(2.25, data[1][4], color = 'darkgrey', width = 0.25)
ax.bar(2.50, data[2][4], color = 'orange', width = 0.25)

ax.bar(3.00, data[0][5], color = 'lightgrey', width = 0.25)
ax.bar(3.25, data[1][5], color = 'darkgrey', width = 0.25)
ax.bar(3.50, data[2][5], color = 'orange', width = 0.25)